In [3]:
# import libraries
import pandas as pd
from spotlib import search_song, get_audio_features, add_audio_features, recommend_song
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle


In [4]:
# load CSVs
hot_100 = pd.read_csv('extended_hot_100.csv')
not_hot = pd.read_csv('extended_not_hot.csv')

# merge the frames
songs_df = pd.concat([hot_100, not_hot], ignore_index=True)
songs_df = songs_df.dropna()

# define the features we are using
features = ['danceability',  'valence']
songs_features = songs_df[features]

# load the saved scaler and KMeans model
scaler_filename_pickle = 'scaler_pickle.pkl'
kmeans_filename_pickle = 'kmeans_pickle.pkl'

with open(scaler_filename_pickle, 'rb') as scaler_file:
    loaded_scaler = pickle.load(scaler_file)

with open(kmeans_filename_pickle, 'rb') as kmeans_file:
    loaded_kmeans = pickle.load(kmeans_file)

# define song clusters
songs_features_scaled_clean = loaded_scaler.transform(songs_features)
songs_clusters = loaded_kmeans.predict(songs_features_scaled_clean)

# add cluster column to dataframe
songs_df_clean = songs_df.dropna()
songs_df_clean['cluster'] = songs_clusters





In [ ]:
# song recommender

with open(scaler_filename_pickle, 'rb') as scaler_file:
    loaded_scaler = pickle.load(scaler_file)

with open(kmeans_filename_pickle, 'rb') as kmeans_file:
    loaded_kmeans = pickle.load(kmeans_file)

def recommend_song(title, artist):
    # get song ID
    song_id = search_song(title, artist)
    if song_id == "not found":
        print("Song not found in Spotify.")
        return
    
    # get audio features
    song_features_df = get_audio_features([song_id])
    song_features_selected = song_features_df[features]
    
    # scale the audio features using loaded_scaler
    song_features_scaled = loaded_scaler.transform(song_features_selected)
    
    # determine cluster using loaded_kmeans
    cluster = loaded_kmeans.predict(song_features_scaled)
    
    # return a song from the same cluster
    matched_songs = songs_df[songs_clusters == cluster[0]]
    
    # Get a random song from the matched songs
    recommended_song = matched_songs.sample()
    
    return recommended_song[['Song', 'Artist']], song_id

while True:  # loop to keep prompting the user for new songs
    title = input("Enter the title of the song (or 'exit' to quit): ")
    
    # check if user wants to exit
    if title.lower() == 'exit':
        break
    #input the song name
    artist = input("Enter the artist of the song: ")
    recommended, song_id = recommend_song(title, artist)  # Unpack both recommended song and song_id
    
    if recommended is not None:  # Only display the recommendation if one is returned
        print("\nRecommended song for you:")
        print(recommended)
        print(f"Listen to the song here: https://open.spotify.com/track/{song_id}")
    print("\n")  # Adding a newline for cleaner output separation



In [ ]:
def recommend_song(title, artist):
    # get song ID
    input_song_id = search_song(title, artist)
    if input_song_id == "not found":
        print("Song not found in Spotify.")
        return
    
    # get audio features
    song_features_df = get_audio_features([input_song_id])
    song_features_selected = song_features_df[features]
    
    # scale the audio features using loaded_scaler
    song_features_scaled = loaded_scaler.transform(song_features_selected)
    
    # determine cluster using loaded_kmeans
    cluster = loaded_kmeans.predict(song_features_scaled)
    
    # return a song from the same cluster
    matched_songs = songs_df[songs_clusters == cluster[0]]
    
    # Get a random song from the matched songs
    recommended_song = matched_songs.sample()
    
    # Get the Spotify ID of the recommended song
    recommended_song_id = recommended_song['id'].values[0]  # Assuming you have an 'id' column in your dataframe
    
    return recommended_song[['Song', 'Artist']], recommended_song_id

while True:
    title = input("Enter the title of the song (or 'exit' to quit): ")
    
    # check if user wants to exit
    if title.lower() == 'exit':
        break
    #input the song name
    artist = input("Enter the artist of the song: ")
    
    recommended, recommended_song_id = recommend_song(title, artist)
    
    if recommended is not None: 
        print("\nRecommended song for you:")
        print(recommended)
        print(f"Listen to the recommended song here: https://open.spotify.com/track/{recommended_song_id}")
    print("\n")



Enter the title of the song (or 'exit' to quit): rythm is a dancer
Enter the artist of the song: snap

Recommended song for you:
                                                   Song        Artist
2541  Um Velho Malandro de Corpo Fechado - Da Melhor...  Arlindo Cruz
Listen to the recommended song here: https://open.spotify.com/track/50akPi1b14uMI3SkZRIPW0


